In [10]:
#存入URL
import requests
from bs4 import BeautifulSoup as bs
import string
from math import ceil
headers = {'cookie':'TRID_G=8711a259-e871-4895-bfc8-ef664f999722; SEID_G=bf7c90c9-369e-4672-9cb5-8c62da63d998; ASP.NET_SessionId=pos1kcp2zp0elm1vbhb34ujn; _ga=GA1.3.988170899.1438438756; WMX_Channel=,3,; __ltmwga=utmcsr=(direct)|utmcmd=(none); _gat_UA-22823074-5=1; CSCWORM_UNID=1440421905627627; _dc_gtm_UA-22823074-5=1; _dc_gtm_UA-34471860-1=1; _gat_UA-34471860-1=1; objID=; sto-id-%3Fv30%3FGroup-buy.housefun.com.tw%3A80=EFABBOKM; _ga=GA1.4.988170899.1438438756; __asc=5fa575a314f5fd6171a385af309; __auc=3ce783c214ee9a18edcba1d0ab5'}
count = requests.get('http://buy.housefun.com.tw/ashx/buy/new/GetBuyListTabCaseCount.ashx?callback=angular.callbacks._0&RequestPackage=%7B%22Method%22:%22INQUIRE%22,%22Data%22:%7B%22SID%22:%2282628802%22%7D%7D',headers=headers)
soup = bs(count.text)
cases = ''.join(soup.text.split('\"')[25].split(','))
page = int(ceil(float(cases)/10))

f = open('housefun_URL1.txt','w')
page_html = "http://buy.housefun.com.tw/%E8%B2%B7%E5%B1%8B/%E5%8F%B0%E5%8C%97%E5%B8%82?hd_CityID=0000&hd_Purpose=K&hd_Sequence=SEQUP&hd_SearchGroup=Group01&hd_PM={0}&hd_Tab=1&hd_SID=82628774"
for pg in range(1,page+1):
    res = requests.get(page_html.format(pg),headers=headers)
    soup = bs(res.text)
    for i in range(0,10):
        f.write("http://buy.housefun.com.tw" + soup.select('.m-list-obj')[i].select('a')[0]['href']+'\n')
f.close()

KeyboardInterrupt: 

In [13]:
#存入各物件html
import requests
headers = {'cookie':'TRID_G=8711a259-e871-4895-bfc8-ef664f999722; SEID_G=bf7c90c9-369e-4672-9cb5-8c62da63d998; ASP.NET_SessionId=pos1kcp2zp0elm1vbhb34ujn; _ga=GA1.3.988170899.1438438756; WMX_Channel=,3,; __ltmwga=utmcsr=(direct)|utmcmd=(none); _gat_UA-22823074-5=1; CSCWORM_UNID=1440421905627627; _dc_gtm_UA-22823074-5=1; _dc_gtm_UA-34471860-1=1; _gat_UA-34471860-1=1; objID=; sto-id-%3Fv30%3FGroup-buy.housefun.com.tw%3A80=EFABBOKM; _ga=GA1.4.988170899.1438438756; __asc=5fa575a314f5fd6171a385af309; __auc=3ce783c214ee9a18edcba1d0ab5'}
dir = 'housefun'
f = open("housefun_URL.txt","r")
for line in f:
    file_name = line.strip().split('/')[5].split('.')[0]
    input_file = open(dir+'/'+file_name+'.txt','w')
    
    res = requests.get(line.strip(),headers=headers)
    response = res.text.encode('utf8')
    input_file.write(response)
f.close()

KeyboardInterrupt: 

In [62]:
import requests
from bs4 import BeautifulSoup
import re
import glob
import os
from math import ceil

    #ff = open('housefun_detail.txt','w')
dir_path = '/Users/Derek/Project@iii/housefun/'
for file_name in glob.glob(os.path.join(dir_path, "*.txt")):
    f = open(file_name, "r")
    soup = bs(f.read())
    
    pk = soup.select('#query-hf-id')[0]['value']
    latitude = soup.select('#query-lat')[0]['value']
    longitude = soup.select('#query-lng')[0]['value']
    price = ''.join(soup.select('#total-amount')[0].text.split(','))
    
    string = ' '.join(soup.select('.info-sub')[0].text.split())
    
#格局
    Housing_pattern_bedroom = re.search('([\d]*)房',str(string.encode('utf8'))).group(1)
    if re.search('([\d]*)廳',str(string.encode('utf8'))):
        Housing_pattern_hall = re.search('([\d]*)廳',str(string.encode('utf8'))).group(1)
    else:Housing_pattern_hall = ''
    Housing_pattern_toilet = re.search('([\d\.]*)衛',str(string.encode('utf8'))).group(1)
#總坪數
    area_total = re.search('總坪數([\d\.]*) ',str(string.encode('utf8'))).group(1)
#每坪單價    
    if re.search('單價([\d\.]*)萬',str(string.encode('utf8'))):
        price_per = re.search('單價([\d\.]*)萬',str(string.encode('utf8'))).group(1)  
    else: price_per = ceil(float(price)/float(area_total))
    
    road = soup.select('.extra-info')[0].select('li')[0].text
    
    extra = ''
    for ele in soup.select('.extra-info')[0].select('li'):
        extra = extra + ' ' + ' '.join(ele.text.split())
        
#樓層/樓高
    if re.search('樓層 ([\d]*)',str(extra.encode('utf8'))):
        floor = re.search('樓層 ([\d]*)',str(extra.encode('utf8'))).group(1)
    elif re.search('樓層 ([\S]*)',str(extra.encode('utf8'))):
        floor = re.search('樓層 ([\S]*)',str(extra.encode('utf8'))).group(1)
    else: floor = ''
        
    if re.search('/ ([\d]*)',str(extra.encode('utf8'))):
        building_height = re.search('/ ([\d]*)',str(extra.encode('utf8'))).group(1)
    else: building_height = ''
#屋齡
    if re.search('屋齡 ([\d]*)',str(extra.encode('utf8'))):
        age = re.search('屋齡 ([\d]*)',str(extra.encode('utf8'))).group(1)
    else: age = ''
#類型
    if re.search('公寓',str(extra.encode('utf8'))):
        building_type = unicode('公寓','utf8')
    elif re.search('大樓',str(extra.encode('utf8'))):
        building_type = unicode('大樓','utf8')
    elif re.search('透天厝',str(extra.encode('utf8'))):
        building_type = unicode('透天厝','utf8')
    else: building_type =''
#方位
    if re.search('朝向([\S]*) ',str(extra.encode('utf8'))): 
        orientation = unicode(re.search('朝向([\S]*) ',str(extra.encode('utf8'))).group(1),'utf8')
    else: orientation = ' '
    
    print pk+','+price+','+latitude+','+longitude+','+road+','+floor+','+building_height+','+age+','+building_type+','+orientation


1281601,560,24.990214,121.54769,台北市文山區試院路,4,5,23,大樓,北
1494973,6230,25.0529272,121.5198509,台北市大同區南京西路,9,13,9,大樓, 
1547272,3180,24.9863488,121.5890677,台北市文山區萬壽路,1,2,41,透天厝,北
1731933,1966,25.0371611,121.5829359,台北市信義區福德街,2,4,42,公寓,西南
1734766,1980,25.0482033,121.541357,台北市中山區長安東路二段,7,12,32,大樓,北
1736705,1680,25.0606675468729,121.533783769312,台北市中山區民權東路二段,4,4,45,公寓,北
512550,1050,25.0019899471606,121.557269821705,台北市文山區興德路,5,5,37,公寓, 


In [7]:
#計算URL
line_num = 0
f = open("housefun_1.txt","r")
for line in f.readlines():
    line_num = line_num + 1
print line_num
f.close()

5235


In [69]:
#將重複的物件讀出
case_dic = {}
f = open('housefun_URL.txt','r')
re = open('re_3.txt','w')
for line in f.readlines():
    rec = line.strip()
    caseno = rec.split('/')[5]
    if caseno not in case_dic:
        case_dic[caseno] = 1
    else:
        re.write(caseno+'\n')
re.close()
f.close()